In [1]:
import requests
import numpy as np
import pandas as pd
import datetime
from dotenv import load_dotenv

In [2]:
load_dotenv('.env')
fmp_key = os.getenv('FMP_API')

type(fmp_key)

str

In [3]:
company = 'PG'

inc_stmt = requests.get(f'https://financialmodelingprep.com/api/v3/income-statement/{company}?apikey={fmp_key}').json()
bal_sht = requests.get(f'https://financialmodelingprep.com/api/v3/balance-sheet-statement/{company}?apikey={fmp_key}').json()
bal_sht

[{'date': '2020-06-30',
  'symbol': 'PG',
  'reportedCurrency': 'USD',
  'fillingDate': '2020-08-06',
  'acceptedDate': '2020-08-06 16:17:03',
  'period': 'FY',
  'cashAndCashEquivalents': 16181000000,
  'shortTermInvestments': 0.0,
  'cashAndShortTermInvestments': 16181000000,
  'netReceivables': 4178000000,
  'inventory': 5498000000,
  'otherCurrentAssets': 0.0,
  'totalCurrentAssets': 27987000000,
  'propertyPlantEquipmentNet': 20692000000,
  'goodwill': 39901000000,
  'intangibleAssets': 23792000000,
  'goodwillAndIntangibleAssets': 63693000000,
  'longTermInvestments': 0.0,
  'taxAssets': 0.0,
  'otherNonCurrentAssets': 8328000000,
  'totalNonCurrentAssets': 92713000000,
  'otherAssets': 0.0,
  'totalAssets': 120700000000,
  'accountPayables': 12071000000,
  'shortTermDebt': 11183000000,
  'taxPayables': 0.0,
  'deferredRevenue': 0.0,
  'otherCurrentLiabilities': 3105000000,
  'totalCurrentLiabilities': 32976000000,
  'longTermDebt': 23537000000,
  'deferredRevenueNonCurrent': 0.0

In [5]:
inc_stmt

[{'date': '2020-06-30',
  'symbol': 'PG',
  'reportedCurrency': 'USD',
  'fillingDate': '2020-08-06',
  'acceptedDate': '2020-08-06 16:17:03',
  'period': 'FY',
  'revenue': 70950000000,
  'costOfRevenue': 35250000000,
  'grossProfit': 35700000000,
  'grossProfitRatio': 0.5031712473572939,
  'researchAndDevelopmentExpenses': 0.0,
  'generalAndAdministrativeExpenses': 19994000000,
  'sellingAndMarketingExpenses': 0.0,
  'otherExpenses': 438000000,
  'operatingExpenses': 19994000000,
  'costAndExpenses': 55244000000,
  'interestExpense': 465000000,
  'depreciationAndAmortization': 3013000000,
  'ebitda': 18719000000,
  'ebitdaratio': 0.26383368569415083,
  'operatingIncome': 15706000000,
  'operatingIncomeRatio': 0.22317124735729388,
  'totalOtherIncomeExpensesNet': 438000000,
  'incomeBeforeTax': 15834000000,
  'incomeBeforeTaxRatio': 0.22317124735729388,
  'incomeTaxExpense': 2731000000,
  'netIncome': 13027000000,
  'netIncomeRatio': 0.18360817477096547,
  'eps': 5.13,
  'epsdiluted':

In [7]:
avg_rev_growth = ((1+(inc_stmt[0]['revenue'] - inc_stmt[1]['revenue']) / inc_stmt[1]['revenue']) * (1+(inc_stmt[1]['revenue'] - inc_stmt[2]['revenue']) / inc_stmt[2]['revenue']) * (1+(inc_stmt[2]['revenue'] - inc_stmt[3]['revenue']) / inc_stmt[3]['revenue']) * (1+(inc_stmt[3]['revenue'] - inc_stmt[4]['revenue']) / inc_stmt[4]['revenue']))**(1/4) -1

net_income = inc_stmt[0]['netIncome']

avg_rev_growth

0.020966441467535457

In [11]:
current_inc_stmt_df = pd.DataFrame.from_dict(inc_stmt[0], orient='index')
current_inc_stmt_df = current_inc_stmt_df[6:27]
current_inc_stmt_df.columns = ['2020 Current']
current_inc_stmt_df['Pct of Revenue'] = current_inc_stmt_df / current_inc_stmt_df.iloc[0]
current_inc_stmt_df['2019'] = pd.DataFrame.from_dict(inc_stmt[1], orient='index')
current_inc_stmt_df['2018'] = pd.DataFrame.from_dict(inc_stmt[2], orient='index')
current_inc_stmt_df['2017'] = pd.DataFrame.from_dict(inc_stmt[3], orient='index')
current_inc_stmt_df['2016'] = pd.DataFrame.from_dict(inc_stmt[4], orient='index')
complete_inc_stmt_df = current_inc_stmt_df[['2016', '2017', '2018', '2019', '2020 Current', 'Pct of Revenue']]
complete_inc_stmt_df 

,2016,2017,2018,2019,2020 Current,Pct of Revenue
revenue,65299000000,65058000000,66832000000,67684000000,70950000000,1
costOfRevenue,32909000000,32535000000,34268000000,34768000000,35250000000,0.496829
grossProfit,32390000000,32523000000,32564000000,32916000000,35700000000,0.503171
grossProfitRatio,0.496026,0.499908,0.487252,0.486319,0.503171,7.09191e-12
researchAndDevelopmentExpenses,0,0,0,0,0,0
generalAndAdministrativeExpenses,18949000000,18568000000,18853000000,19084000000,19994000000,0.281804
sellingAndMarketingExpenses,0,0,0,0,0,0
otherExpenses,325000000,-404000000,-126000000,871000000,438000000,0.00617336
operatingExpenses,18949000000,18568000000,18853000000,19084000000,19994000000,0.281804
costAndExpenses,51858000000,51103000000,53121000000,53852000000,55244000000,0.778633


In [12]:
complete_inc_stmt_df['2021 est'] = (complete_inc_stmt_df['2020 Current']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2022 est'] = (complete_inc_stmt_df['2021 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2023 est'] = (complete_inc_stmt_df['2022 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2024 est'] = (complete_inc_stmt_df['2023 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df['2025 est'] = (complete_inc_stmt_df['2024 est']['revenue'] * (1+avg_rev_growth)) * complete_inc_stmt_df['Pct of Revenue']
complete_inc_stmt_df

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,2016,2017,2018,2019,2020 Current,Pct of Revenue,2021 est,2022 est,2023 est,2024 est,2025 est
revenue,65299000000,65058000000,66832000000,67684000000,70950000000,1,7.24376e+10,7.39563e+10,7.55069e+10,7.709e+10,7.87063e+10
costOfRevenue,32909000000,32535000000,34268000000,34768000000,35250000000,0.496829,3.59891e+10,3.67436e+10,3.7514e+10,3.83005e+10,3.91036e+10
grossProfit,32390000000,32523000000,32564000000,32916000000,35700000000,0.503171,3.64485e+10,3.72127e+10,3.79929e+10,3.87895e+10,3.96028e+10
grossProfitRatio,0.496026,0.499908,0.487252,0.486319,0.503171,7.09191e-12,0.513721,0.524492,0.535489,0.546716,0.558179
researchAndDevelopmentExpenses,0,0,0,0,0,0,0,0,0,0,0
generalAndAdministrativeExpenses,18949000000,18568000000,18853000000,19084000000,19994000000,0.281804,2.04132e+10,2.08412e+10,2.12782e+10,2.17243e+10,2.21798e+10
sellingAndMarketingExpenses,0,0,0,0,0,0,0,0,0,0,0
otherExpenses,325000000,-404000000,-126000000,871000000,438000000,0.00617336,4.47183e+08,4.56559e+08,4.66132e+08,4.75905e+08,4.85883e+08
operatingExpenses,18949000000,18568000000,18853000000,19084000000,19994000000,0.281804,2.04132e+10,2.08412e+10,2.12782e+10,2.17243e+10,2.21798e+10
costAndExpenses,51858000000,51103000000,53121000000,53852000000,55244000000,0.778633,5.64023e+10,5.75848e+10,5.87922e+10,6.00248e+10,6.12833e+10


In [15]:
current_bal_sht_df = pd.DataFrame.from_dict(bal_sht[0],orient='index')
current_bal_sht_df = current_bal_sht_df[6:-2]
current_bal_sht_df.columns = ['2020 Current']
current_bal_sht_df['Pct of Revenue'] = current_bal_sht_df / complete_inc_stmt_df['2020 Current'].iloc[0]
current_bal_sht_df['2019'] = pd.DataFrame.from_dict(bal_sht[1],orient='index')
current_bal_sht_df['2018'] = pd.DataFrame.from_dict(bal_sht[2],orient='index')
current_bal_sht_df['2017'] = pd.DataFrame.from_dict(bal_sht[3],orient='index')
current_bal_sht_df['2016'] = pd.DataFrame.from_dict(bal_sht[4],orient='index')
complete_bal_sht_df = current_bal_sht_df
complete_bal_sht_df[['2016', '2017', '2018', '2019', '2020 Current', 'Pct of Revenue']]

,2016,2017,2018,2019,2020 Current,Pct of Revenue
cashAndCashEquivalents,7102000000,5569000000,2569000000,4239000000,16181000000,0.228062
shortTermInvestments,6246000000,9568000000,9281000000,6048000000,0,0
cashAndShortTermInvestments,13348000000,15137000000,11850000000,10287000000,16181000000,0.228062
netReceivables,4373000000,4594000000,4686000000,4951000000,4178000000,0.0588865
inventory,4716000000,4624000000,4738000000,5017000000,5498000000,0.0774912
otherCurrentAssets,0,0,0,0,0,0
totalCurrentAssets,33782000000,26494000000,23320000000,22473000000,27987000000,0.394461
propertyPlantEquipmentNet,19385000000,19893000000,20600000000,21271000000,20692000000,0.291642
goodwill,44350000000,44699000000,45175000000,40273000000,39901000000,0.562382
intangibleAssets,24527000000,24187000000,23902000000,24215000000,23792000000,0.335335


In [17]:
complete_bal_sht_df['2021 est'] = complete_inc_stmt_df['2021 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2022 est'] = complete_inc_stmt_df['2022 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2023 est'] = complete_inc_stmt_df['2023 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2024 est'] = complete_inc_stmt_df['2024 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df['2025 est'] = complete_inc_stmt_df['2025 est']['revenue'] * complete_bal_sht_df['Pct of Revenue']
complete_bal_sht_df

,2020 Current,Pct of Revenue,2019,2018,2017,2016,2021 est,2022 est,2023 est,2024 est,2025 est
cashAndCashEquivalents,16181000000,0.228062,4239000000,2569000000,5569000000,7102000000,1.65203e+10,1.68666e+10,1.72203e+10,1.75813e+10,1.79499e+10
shortTermInvestments,0,0,6048000000,9281000000,9568000000,6246000000,0,0,0,0,0
cashAndShortTermInvestments,16181000000,0.228062,10287000000,11850000000,15137000000,13348000000,1.65203e+10,1.68666e+10,1.72203e+10,1.75813e+10,1.79499e+10
netReceivables,4178000000,0.0588865,4951000000,4686000000,4594000000,4373000000,4.2656e+09,4.35503e+09,4.44634e+09,4.53957e+09,4.63474e+09
inventory,5498000000,0.0774912,5017000000,4738000000,4624000000,4716000000,5.61327e+09,5.73096e+09,5.85112e+09,5.9738e+09,6.09905e+09
otherCurrentAssets,0,0,0,0,0,0,0,0,0,0,0
totalCurrentAssets,27987000000,0.394461,22473000000,23320000000,26494000000,33782000000,2.85738e+10,2.91729e+10,2.97845e+10,3.0409e+10,3.10466e+10
propertyPlantEquipmentNet,20692000000,0.291642,21271000000,20600000000,19893000000,19385000000,2.11258e+10,2.15688e+10,2.2021e+10,2.24827e+10,2.29541e+10
goodwill,39901000000,0.562382,40273000000,45175000000,44699000000,44350000000,4.07376e+10,4.15917e+10,4.24637e+10,4.3354e+10,4.4263e+10
intangibleAssets,23792000000,0.335335,24215000000,23902000000,24187000000,24527000000,2.42908e+10,2.48001e+10,2.53201e+10,2.5851e+10,2.6393e+10


In [25]:
projected_cf = {}

projected_cf['2021 est'] = {}

projected_cf['2021 est']['Net Income'] = complete_inc_stmt_df['2021 est']['netIncome']
projected_cf['2021 est']['Depreciation'] = complete_inc_stmt_df['2021 est']['depreciationAndAmortization']
projected_cf['2021 est']['Change in Receivables'] = complete_bal_sht_df['2020 Current']['netReceivables'] - complete_bal_sht_df['2021 est']['netReceivables']
projected_cf['2021 est']['Change in Inventory'] = complete_bal_sht_df['2020 Current']['inventory'] - complete_bal_sht_df['2021 est']['inventory']
projected_cf['2021 est']['Change in Payables'] = complete_bal_sht_df['2021 est']['accountPayables'] - complete_bal_sht_df['2020 Current']['accountPayables']
projected_cf['2021 est']['Change in NWC'] = projected_cf['2021 est']['Change in Receivables'] + projected_cf['2021 est']['Change in Inventory'] + projected_cf['2021 est']['Change in Payables']
projected_cf['2021 est']['OP CF'] = projected_cf['2021 est']['Net Income'] + projected_cf['2021 est']['Depreciation'] + projected_cf['2021 est']['Change in NWC']
projected_cf['2021 est']['CAPEX'] = complete_bal_sht_df['2021 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2020 Current']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2021 est']['depreciationAndAmortization']
projected_cf['2021 est']['Unlev FCF'] = projected_cf['2021 est']['CAPEX'] + projected_cf['2021 est']['OP CF']


projected_cf['2022 est'] = {}

projected_cf['2022 est']['Net Income'] = complete_inc_stmt_df['2022 est']['netIncome']
projected_cf['2022 est']['Depreciation'] = complete_inc_stmt_df['2022 est']['depreciationAndAmortization']
projected_cf['2022 est']['Change in Receivables'] = complete_bal_sht_df['2021 est']['netReceivables'] - complete_bal_sht_df['2022 est']['netReceivables']
projected_cf['2022 est']['Change in Inventory'] = complete_bal_sht_df['2021 est']['inventory'] - complete_bal_sht_df['2022 est']['inventory']
projected_cf['2022 est']['Change in Payables'] = complete_bal_sht_df['2022 est']['accountPayables'] - complete_bal_sht_df['2021 est']['accountPayables']
projected_cf['2022 est']['Change in NWC'] = projected_cf['2022 est']['Change in Receivables'] + projected_cf['2022 est']['Change in Inventory'] + projected_cf['2022 est']['Change in Payables']
projected_cf['2022 est']['OP CF'] = projected_cf['2022 est']['Net Income'] + projected_cf['2022 est']['Depreciation'] + projected_cf['2022 est']['Change in NWC']
projected_cf['2022 est']['CAPEX'] = complete_bal_sht_df['2022 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2021 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2022 est']['depreciationAndAmortization']
projected_cf['2022 est']['Unlev FCF'] = projected_cf['2022 est']['CAPEX'] + projected_cf['2022 est']['OP CF']


projected_cf['2023 est'] = {}

projected_cf['2023 est']['Net Income'] = complete_inc_stmt_df['2023 est']['netIncome']
projected_cf['2023 est']['Depreciation'] = complete_inc_stmt_df['2023 est']['depreciationAndAmortization']
projected_cf['2023 est']['Change in Receivables'] = complete_bal_sht_df['2022 est']['netReceivables'] - complete_bal_sht_df['2023 est']['netReceivables']
projected_cf['2023 est']['Change in Inventory'] = complete_bal_sht_df['2022 est']['inventory'] - complete_bal_sht_df['2023 est']['inventory']
projected_cf['2023 est']['Change in Payables'] = complete_bal_sht_df['2023 est']['accountPayables'] - complete_bal_sht_df['2022 est']['accountPayables']
projected_cf['2023 est']['Change in NWC'] = projected_cf['2023 est']['Change in Receivables'] + projected_cf['2023 est']['Change in Inventory'] + projected_cf['2023 est']['Change in Payables']
projected_cf['2023 est']['OP CF'] = projected_cf['2023 est']['Net Income'] + projected_cf['2023 est']['Depreciation'] + projected_cf['2023 est']['Change in NWC']
projected_cf['2023 est']['CAPEX'] = complete_bal_sht_df['2023 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2022 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2023 est']['depreciationAndAmortization']
projected_cf['2023 est']['Unlev FCF'] = projected_cf['2023 est']['CAPEX'] + projected_cf['2023 est']['OP CF']


projected_cf['2024 est'] = {}

projected_cf['2024 est']['Net Income'] = complete_inc_stmt_df['2024 est']['netIncome']
projected_cf['2024 est']['Depreciation'] = complete_inc_stmt_df['2024 est']['depreciationAndAmortization']
projected_cf['2024 est']['Change in Receivables'] = complete_bal_sht_df['2023 est']['netReceivables'] - complete_bal_sht_df['2024 est']['netReceivables']
projected_cf['2024 est']['Change in Inventory'] = complete_bal_sht_df['2023 est']['inventory'] - complete_bal_sht_df['2024 est']['inventory']
projected_cf['2024 est']['Change in Payables'] = complete_bal_sht_df['2024 est']['accountPayables'] - complete_bal_sht_df['2023 est']['accountPayables']
projected_cf['2024 est']['Change in NWC'] = projected_cf['2024 est']['Change in Receivables'] + projected_cf['2024 est']['Change in Inventory'] + projected_cf['2024 est']['Change in Payables']
projected_cf['2024 est']['OP CF'] = projected_cf['2024 est']['Net Income'] + projected_cf['2024 est']['Depreciation'] + projected_cf['2024 est']['Change in NWC']
projected_cf['2024 est']['CAPEX'] = complete_bal_sht_df['2024 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2023 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2024 est']['depreciationAndAmortization']
projected_cf['2024 est']['Unlev FCF'] = projected_cf['2024 est']['CAPEX'] + projected_cf['2024 est']['OP CF']


projected_cf['2025 est'] = {}

projected_cf['2025 est']['Net Income'] = complete_inc_stmt_df['2025 est']['netIncome']
projected_cf['2025 est']['Depreciation'] = complete_inc_stmt_df['2025 est']['depreciationAndAmortization']
projected_cf['2025 est']['Change in Receivables'] = complete_bal_sht_df['2024 est']['netReceivables'] - complete_bal_sht_df['2025 est']['netReceivables']
projected_cf['2025 est']['Change in Inventory'] = complete_bal_sht_df['2024 est']['inventory'] - complete_bal_sht_df['2025 est']['inventory']
projected_cf['2025 est']['Change in Payables'] = complete_bal_sht_df['2025 est']['accountPayables'] - complete_bal_sht_df['2024 est']['accountPayables']
projected_cf['2025 est']['Change in NWC'] = projected_cf['2025 est']['Change in Receivables'] + projected_cf['2025 est']['Change in Inventory'] + projected_cf['2025 est']['Change in Payables']
projected_cf['2025 est']['OP CF'] = projected_cf['2025 est']['Net Income'] + projected_cf['2025 est']['Depreciation'] + projected_cf['2025 est']['Change in NWC']
projected_cf['2025 est']['CAPEX'] = complete_bal_sht_df['2025 est']['propertyPlantEquipmentNet'] - complete_bal_sht_df['2024 est']['propertyPlantEquipmentNet'] + complete_inc_stmt_df['2025 est']['depreciationAndAmortization']
projected_cf['2025 est']['Unlev FCF'] = projected_cf['2025 est']['CAPEX'] + projected_cf['2025 est']['OP CF']

projected_cf

{'2021 est': {'Net Income': 13300129832.997585,
  'Depreciation': 3076171888.1416845,
  'Change in Receivables': -87597792.45136309,
  'Change in Inventory': -115273495.18850994,
  'Change in Payables': 253085914.95462227,
  'Change in NWC': 50214627.31474924,
  'OP CF': 16426516348.45402,
  'CAPEX': 3510009494.9879293,
  'Unlev FCF': 19936525843.441948},
 '2022 est': {'Net Income': 13578986226.65175,
  'Depreciation': 3140668265.978485,
  'Change in Receivables': -89434406.43947935,
  'Change in Inventory': -117690370.17813778,
  'Change in Payables': 258392225.9767723,
  'Change in NWC': 51267449.35915518,
  'OP CF': 16770921941.989391,
  'CAPEX': 3583601903.6150856,
  'Unlev FCF': 20354523845.604477},
 '2023 est': {'Net Income': 13863689246.561314,
  'Depreciation': 3206516903.346069,
  'Change in Receivables': -91309527.68727684,
  'Change in Inventory': -120157918.43577003,
  'Change in Payables': 263809791.45838165,
  'Change in NWC': 52342345.33533478,
  'OP CF': 17122548495.242

In [29]:
cf_forecast = pd.DataFrame.from_dict(projected_cf, orient='columns')
pd.options.display.float_format = '{:,.0f}'.format
cf_forecast = cf_forecast.reindex(['Net Income', 'Depreciation', 'Change in Receivables','Change in Inventory', 'Change in Payables', 'Change in NWC', 'OP CF', 'CAPEX', 'Unlev FCF'])

cf_forecast


,2021 est,2022 est,2023 est,2024 est,2025 est
Net Income,"13,300,129,833","13,578,986,227","13,863,689,247","14,154,361,476","14,451,128,067"
Depreciation,"3,076,171,888","3,140,668,266","3,206,516,903","3,273,746,152","3,342,384,959"
Change in Receivables,"-87,597,792","-89,434,406","-91,309,528","-93,223,964","-95,178,538"
Change in Inventory,"-115,273,495","-117,690,370","-120,157,918","-122,677,202","-125,249,307"
Change in Payables,"253,085,915","258,392,226","263,809,791","269,340,944","274,988,065"
Change in NWC,"50,214,627","51,267,449","52,342,345","53,439,778","54,560,220"
OP CF,"16,426,516,348","16,770,921,942","17,122,548,495","17,481,547,406","17,848,073,246"
CAPEX,"3,510,009,495","3,583,601,904","3,658,737,283","3,735,447,984","3,813,767,036"
Unlev FCF,"19,936,525,843","20,354,523,846","20,781,285,778","21,216,995,390","21,661,840,282"
